In [2]:
import pandas as pd
import json
import urllib
import re
import datetime
from datetime import datetime
import pytz
import matplotlib.pyplot as plt
import io
from pandas.io.json import json_normalize
import pandas as pd
import os
import csv
import operator
import itertools

In [3]:
with open('0728id_xy_version2.csv') as f:
    reader = csv.DictReader(f)
    rows = list(reader)

with open('0728id_xy_version2.json', 'w') as f:
    json.dump(rows, f)

In [4]:
with open ('0728id_xy_version2.json','r', encoding='utf-8',errors='replace') as myfile:
    data = myfile.readlines()
    ind_data=[json.loads(data[i]) for i in range(len(data))]

In [5]:
with open('0728id_xy_version2.json') as f:
    things = [json.loads(line) for line in f]

In [6]:
new_things = {}
for thing in things:
    thing_id = thing[0]['id']
    try:
        old_thing = new_things[thing_id]
    except KeyError:
        new_things[thing_id] = thing
    else:
        old_thing['x'].extend(thing['x'])
        old_thing['y'].extend(thing['y'])
        old_thing['timestamps'].extend(thing['timestamps'])
new_things = new_things.values()

In [20]:
def merge(things):
    return {'id': things[0]['id'],
            'x': list(itertools.chain(float(t['x']) for t in things)),
            'y': list(itertools.chain(float(t['y']) for t in things)),
            'timestamps': list(itertools.chain(float(t['timestamps']) for t in things))}
sorted_things = sorted(things[0], key=operator.itemgetter('id'))
grouped_things = itertools.groupby(sorted_things, key=operator.itemgetter('id'))
agents = [merge(list(group)) for key, group in grouped_things]
x = agents[1]['timestamps']
print (agents[1]['x'])

[65940.0, 95921.0, 85926.0, 155925.0, 75924.0, 65940.0, 65940.0, 95921.0, 128233.0, 85926.0, 95921.0, 75924.0, 95921.0, 128233.0, 128233.0, 75924.0, 85926.0, 85926.0, 65940.0, 75924.0, 128233.0, 85926.0, 65940.0, 95921.0, 128233.0, 65940.0, 75924.0, 85926.0, 75924.0, 75924.0, 75924.0, 95921.0, 95921.0, 85926.0, 128233.0, 65940.0, 75924.0, 75924.0, 95921.0, 75924.0, 128233.0, 65940.0, 85926.0, 85926.0, 75924.0, 75924.0, 128233.0, 85926.0, 65940.0, 95921.0, 128233.0, 65940.0, 85926.0, 95921.0, 65940.0, 95921.0, 85926.0, 85926.0, 65940.0, 85926.0, 95921.0, 75924.0, 75924.0, 128233.0, 128233.0, 85926.0, 128233.0, 85926.0, 75924.0, 128233.0, 65940.0, 85926.0, 85926.0, 128233.0, 85926.0, 75924.0, 75924.0, 85926.0, 195866.0, 85926.0, 128233.0, 65940.0, 85926.0, 75924.0, 75924.0, 85926.0, 75924.0, 65940.0, 85926.0, 75924.0, 65940.0, 65940.0, 65940.0, 65940.0, 85926.0, 75924.0, 85926.0, 65940.0, 65940.0, 95921.0, 85926.0, 85926.0, 65940.0, 128233.0, 65940.0, 95921.0, 75924.0, 85926.0, 65940.0, 

In [26]:
import numpy as np

def maxDistFromGroup(newPoint, pointGroup):
    #takes a point and an arra of points and returns 
    #the max dist of any point in the group from this new point
    maxDist=0
    for p in pointGroup:
        distN=np.sqrt( (p[1] - newPoint[1])**2 + (p[0] - newPoint[0])**2)
        maxDist=max(distN,maxDist)
    return maxDist

def extractStays(agents, maxRoam, minStay):
    #takes a list of agent objects, and returns a list of stays for each agent
    #each agent in the input data should be a dictionary with an id, a list of x coordinates, a list y coordinates and a list of timestamps
    # the lists of coordintes and timestamps must also be sorted from earliest to latest
    #maxRoam is the distance threshold and minStay is the time threshold
    stayData=[]
    for a in range(len(agents)):
        xList, yList, timeList =agents[a]['x'], agents[a]['y'], agents[a]['timestamps']
        stays=[]
        i=0
        while i<len(xList):
            j=i
            while (j+1)<len(xList) and maxDistFromGroup([xList[j+1],yList[j+1]], [[xList[ij],yList[ij]] for ij in range(i,j+1)])<maxRoam:
                j+=1
            #create a stay from the middle position with arrival and departure time
            jj=min(j+1,len(xList)-1)
            # departure time is the timestamp of the next obs after this cluster. Unless this cluster contains the last point in the series
            if (float(timeList[jj])-float(timeList[i]))>minStay:
                stays.extend([{'p':[xList[int((i+j)/2)],yList[int((i+j)/2)]], 'start':timeList[i],'end':timeList[jj], 'n':(j-i+1)}])       
            i=j+1
        if stays: # if stays is empty, no need to add anything
            #add the new agent to the stay data
            stayData.append({'stays':stays, 'id': agents[a]['id']})
    return stayData

if __name__ == '__main__':
    maxRoam=15
    minStay=20
    result = extractStays(agents, maxRoam, minStay)
    print (result[0])

{'stays': [{'p': [65940.0, 133825.0], 'start': 1532750000000.0, 'end': 1532760000000.0, 'n': 2}, {'p': [128233.0, 133065.0], 'start': 1532740000000.0, 'end': 1532760000000.0, 'n': 1}, {'p': [128233.0, 133065.0], 'start': 1532740000000.0, 'end': 1532760000000.0, 'n': 2}, {'p': [65940.0, 133825.0], 'start': 1532740000000.0, 'end': 1532750000000.0, 'n': 1}, {'p': [128233.0, 133065.0], 'start': 1532740000000.0, 'end': 1532760000000.0, 'n': 1}, {'p': [65940.0, 133825.0], 'start': 1532740000000.0, 'end': 1532750000000.0, 'n': 1}, {'p': [65940.0, 133825.0], 'start': 1532740000000.0, 'end': 1532750000000.0, 'n': 1}, {'p': [85926.0, 133467.0], 'start': 1532740000000.0, 'end': 1532760000000.0, 'n': 1}, {'p': [85926.0, 133467.0], 'start': 1532750000000.0, 'end': 1532760000000.0, 'n': 1}, {'p': [75924.0, 133311.0], 'start': 1532750000000.0, 'end': 1532760000000.0, 'n': 2}, {'p': [128233.0, 133065.0], 'start': 1532740000000.0, 'end': 1532760000000.0, 'n': 1}, {'p': [75924.0, 133311.0], 'start': 153